In [303]:
import pandas as pd
import re
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor, LLMChainFilter
from sentence_transformers import CrossEncoder
import dotenv
import os

dotenv.load_dotenv()

True

### 2-1.

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini",
                 temperature=0.,)

embeddings = OpenAIEmbeddings()

In [124]:
path = "./files/rules/rules2/"
filename = os.listdir(path)

In [ ]:
# 텍스트 스플리터 생성

splitter = RecursiveCharacterTextSplitter(chunk_size=100, 
                                          chunk_overlap=10)
                                        
                                        #       separators=["\n\n", "\n"])

In [236]:
# PDF 로더 생성

hr_loader = PyPDFLoader(path+filename[0]).load()
security_loader = PyPDFLoader(path+filename[1]).load()
onboard_loader = PyPDFLoader(path+filename[2]).load()
tools_loader = PyPDFLoader(path+filename[3]).load()
culture_loader = PyPDFLoader(path+filename[4]).load()

In [227]:
def cleaning_page(docs):
    lens = None
    for idx, doc in enumerate(docs):
        corpus = doc.page_content.replace("\xa0", "").replace("  ", " ").split("\n")
        if lens is None:
            lens = []
            for sentence in corpus:
                lens.append(len(sentence))
            length = sorted(lens)[len(lens)//2]
        else:
            pass

        cleaning_corpus = []
        for sentence in corpus[:-2]:
            if len(sentence) >= length:
                cleaning_corpus.append(sentence)
            else:
                cleaning_corpus.append(sentence+"\n\n")   
        docs[idx].page_content = "".join(cleaning_corpus)

    return docs

In [237]:
hr_docs = cleaning_page(hr_loader)
security_docs = cleaning_page(security_loader)
onboard_docs = cleaning_page(onboard_loader)
tools_docs = cleaning_page(tools_loader)
culture_docs = cleaning_page(culture_loader)

In [255]:
# # PDF 문서 로드 및 스플릿

# hr_split_docs = hr_docs.load_and_split(text_splitter=splitter)
# security_docs = security_docs.load_and_split(text_splitter=splitter)
# onboard_docs = onboard_docs.load_and_split(text_splitter=splitter)
# tools_docs = tools_docs.load_and_split(text_splitter=splitter)
# culture_docs = culture_docs.load_and_split(text_splitter=splitter)

In [253]:
hr_split_docs = splitter.split_documents(hr_docs)
security_docs = splitter.split_documents(security_docs)
onboard_docs = splitter.split_documents(onboard_docs)
tools_docs = splitter.split_documents(tools_docs)
culture_docs = splitter.split_documents(culture_docs)

In [254]:
for i in culture_docs:
    print(i.page_content)

5 . 문화 규칙
우리는 딱딱한 규칙보다 , 자연스럽게 몸에 배는 문화를 더 믿습니다 . 정해진 틀 없이 일하지만 , 서로를 배려하고 성장하기 위해 우리 팀만의 ‘ 작은 원칙 ʼ 들을 마음속에
ʼ 들을 마음속에 품고 일합니다 .  누가 시켜서가 아니라 , 우리가 함께 잘 일하기 위해 자주 떠올리는 말들입니다 . 이 말들 속엔 우리가 바라는 팀의 분위기와 , 함께 일하고
, 함께 일하고 싶은 동료의 모습이 담겨 있습니다 .1 . 시간은 약속보다 더 큰 신뢰예요 .
12시는 12 시 . 1 분도 소중하니까요 .
회의 , 외부 미팅 , 마감 등 모든 일정은 정시에 시작하고 끝내는 것을 기본으로 합니다 .유연근무제라도 합의된 시간에는 모두 온전히 참여해야 합니다 .
약속을 어기게 될 경우 사전에 공유하고 대체 방안을 제시합니다 .2 . 호칭은 가볍게 , 태도는 진지하게 . 

직급은 벗고 이름으로 , 대신 일엔 책임감 있게 .
사내 커뮤니케이션은 이름 또는 닉네임을 기본으로 하며 , 직급 중심의 호칭을 지양합니다 .
수평적인 소통을 지향하지만 , 의사결정 구조와 역할 책임은 분명히 따릅니다 .토론과 피드백은 누구나 자유롭게 하되 , 결과에 대해서는 주도자가 책임을 집니다 .
3 . 길게 말하지 말고 , 명확하게 말해요 .

보고는 짧게 , 근거는 정확하게 . 감정보단 데이터 !
주간 보고 , 상황 공유 등은 빠르고 간결하게 전달하되 , 핵심 데이터는 빠짐없이 포함합니다 .
회의 시에는 문제의 원인보다 해결 방안에 집중하고 , 의사소통은 근거를 중심으로 합니다 .
보고를 위한 보고보다는 , 실제 도움이 되는 공유를 추구합니다 .4 . 잡담은 팀의 산소예요 .

쓸데없는 얘기 속에 진짜 아이디어가 숨어 있어요 .
커피챗 , 점심 대화 , 퇴근 후 맥주 한 잔은 팀의 유대감을 쌓는 소중한 시간입니다 .자유로운 의견 교환은 장려하지만 , 험담이나 배제성 농담은 금지합니다 .회의실 외에서 나눈
외에서 나눈 대화도 업무에 도움이 된다면 팀과 자유롭게 공유하세요 .5 . 먼저 본

In [256]:
total_docs = hr_split_docs+security_docs+onboard_docs+tools_docs+culture_docs

In [258]:
# 벡터스토어 생성

total_vector_store = FAISS.from_documents(embedding=embeddings, documents=total_docs)
# hr_vector_store = FAISS.from_documents(embedding=embeddings, documents=hr_split_docs)
# security_vector_store = FAISS.from_documents(embedding=embeddings, documents=security_docs)
# onboard_vector_store = FAISS.from_documents(embedding=embeddings, documents=onboard_docs)
# tools_vector_store = FAISS.from_documents(embedding=embeddings, documents=tools_docs)
# culture_vector_store = FAISS.from_documents(embedding=embeddings, documents=culture_docs)

In [ ]:
# LLM 기반 Reranker

total_retriever = total_vector_store.as_retriever(search_kwargs={"k": 5})
compressor = LLMChainExtractor.from_llm(llm)

In [ ]:
# 모델 로드 (최초 1회 다운로드 후 캐싱됨)
# 점수 기반 Reranker
model = CrossEncoder("BAAI/bge-reranker-base") 

# def rerank(query, docs, top_n=5):
#     pairs = [[query, doc.page_content] for doc in docs]
#     scores = model.predict(pairs)
#     reranked = sorted(zip(scores, docs), key=lambda x: x[0], reverse=True)
#     return [doc for score, doc in reranked[:top_n]]

In [305]:
reranked_retriever = ContextualCompressionRetriever(
    base_retriever=total_retriever,
    base_compressor=compressor
)

In [306]:
query

'연차는 몇일 사용할 수 있나요?'

In [307]:
docs = reranked_retriever.invoke(query)

In [312]:
docs[1].page_content

'연차는 자율적으로 사용하는 것이 원칙이며 , 팀 내 사전 공유만 잘 하면 됩니다 .정시 퇴근도 당연한 권리이며 , 휴가 중인 동료에게는 연락을 삼가는 문화를 지향합니다 .“퇴근해'

In [261]:
query = "연차는 몇일 사용할 수 있나요?"

In [262]:
docs = total_retriever.invoke(query)

In [263]:
pair1 = [docs[0].page_content, query]
pair2 = [docs[1].page_content, query]
pair3 = [docs[2].page_content, query]
pair4 = [docs[3].page_content, query]

In [267]:
pair1

['연차휴가 : 법정 연차휴가 (1 년 15 일 , 이후 근속연수에 따라 가산 ) 를 부여하며 , 별도의 승인 없이 자유롭게 사용 가능합니다 . 휴가 사용 시 동료에게 업무',
 '연차는 몇일 사용할 수 있나요?']

In [264]:
score = model.predict(pair1)

In [265]:
print(f"{score:.8f}")

0.33195952


In [266]:
pair2

['연차는 자율적으로 사용하는 것이 원칙이며 , 팀 내 사전 공유만 잘 하면 됩니다 .정시 퇴근도 당연한 권리이며 , 휴가 중인 동료에게는 연락을 삼가는 문화를 지향합니다 .“퇴근해',
 '연차는 몇일 사용할 수 있나요?']

In [268]:
score = model.predict(pair2)

In [269]:
print(f"{score:.8f}")

0.26285365


In [270]:
pair3

['수습 기간 ( ~ 3 개월 )', '연차는 몇일 사용할 수 있나요?']

In [271]:
score = model.predict(pair3)

In [272]:
print(f"{score:.8f}")

0.00414294


In [273]:
pair4

['연간 200 만원 한도까지 지원받을 수 있습니다 .', '연차는 몇일 사용할 수 있나요?']

In [274]:
score = model.predict(pair4)

In [275]:
print(f"{score:.8f}")

0.00055359


In [67]:
# compressor = CohereRerank(model="rerank-v3.5", top_n=3)

In [276]:
reranked_retriever = ContextualCompressionRetriever(
    base_retriever=total_retriever,
    base_compressor=model
)

ValidationError: 1 validation error for ContextualCompressionRetriever
base_compressor
  Input should be a valid dictionary or instance of BaseDocumentCompressor [type=model_type, input_value=<sentence_transformers.cr...t at 0x0000027802529A20>, input_type=CrossEncoder]
    For further information visit https://errors.pydantic.dev/2.10/v/model_type

In [69]:
prompt = ChatPromptTemplate([
    ("system", "당신은 회사 내규 챗봇입니다. 사용자 정보와 회사 내규 문서가 주어집니다. 그것을 통해 사용자의 행동을 제시하세요.\n"
               "사용자 정보를 고려하여 답변의 수준을 조절하세요."
               "사용자 정보"
               "이름 : {name}\n"
               "직무 : {job}\n"
               "연차 : {exp}\n"
               "---"
               "문서 : {context}\n\n"
               "항상 찾은 근거 문서를 최하단에 함께 제시해주세요.\n"
               "문서에서 응답을 찾을 수 없는 경우 '문서에서 응답을 찾을 수 없습니다.' 라고 답변하세요."), 
    ("user", "{query}")

])

In [70]:
profile = {"name" : "방승욱", "job" : "마케터", "exp" : "1년차"}

In [71]:
def format_docs(docs):

    return "\n\n".join(doc.page_content for doc in docs)

In [72]:
def inject_profile(inputs):
    
    print(inputs)
    
    return {
        **inputs,  # 기존 context + query 그대로 유지
        "name": profile["name"],
        "job": profile["job"],
        "exp": profile["exp"]
    }

In [73]:
# chain 구성

chain = {
    "context": total_retriever | RunnableLambda(format_docs),
    "query": RunnablePassthrough()
} | RunnableLambda(inject_profile) | prompt | llm

In [74]:
query = "재택근무는 언제 할 수 있나요?"

In [75]:
result = chain.invoke(query)

{'context': '자유롭게  할  수  있습니다 . 회사는  “ 휴가나  퇴근  시  눈치  주는  농담을  하지  않는다 ˮ 는  문\n화를  내재화하여  구성원의  정당한  휴식권을  보장합니다 .\n휴가  및  휴직\n연차휴가 : \xa0법정  연차휴가 (1 년  15 일 , 이후  근속연수에  따라  가산 ) 를  부여하며 , 별도의  승\n인  없이  자유롭게  사용  가능합니다 . 휴가  사용  시  동료에게  업무  인수인계를  완료하고 ,  \n회사  인사관리  시스템으로  최소  1 일  전  신청합니다 . 연차는  시간단위  사용도  허용하며\n\n휴가는  충전이에요 . 눈치  주지  말고 , 눈치  보지  말고 .\n연차는  자율적으로  사용하는  것이  원칙이며 , 팀  내  사전  공유만  잘  하면  됩니다 .\n정시  퇴근도  당연한  권리이며 , 휴가  중인  동료에게는  연락을  삼가는  문화를  지향합니다 .\n“퇴근해 ?ˮ , “ 휴가  좋겠네 ~ˮ 같은  농담도  서로  피합니다 .\n7 .  일을  시작할  땐  ‘ 왜 ʼ 부터  물어요 .\n목적이  분명하면 , 방향도  자연스럽게  생겨요 .\n\n하게  초과근무를  할  경우  사전에  팀  리더의  승인을  받아야  합니다 . 초과근무에  대해서는  \n대체휴가  또는  법정  수당을  지급합니다 .\xa0 퇴근  시  눈치주기  문화는  금지이며 , 정시퇴근을  \n1. 인사  운영  메뉴얼\n1\n\n업무  효율과  워라밸을  지원합니다 .\n유연근무제 : 유연근무제는  다음과  같습니다 . 8 시 ~17 시 , 9 시 ~18 시 , 10 시 ~19 시  중  선택\n하여  출근합니다 . 사원증을  태그하였을  때부터  출근으로  인정됩니다 . 다만 , 분  단위  출근\n은  인정되지  않습니다 . ( 예 : 8 시  39 분  출근  → 9 시  출근으로  인정 )\n초과근무 : 불필요한  야근을  지양하며  업무  효율  향상을  추구합니다 . 업무  

In [76]:
print(result.content)

문서에서 재택근무에 대한 언급이 없습니다. 따라서, 재택근무에 대한 정보는 제공할 수 없습니다. 

문서에서 응답을 찾을 수 없습니다.


In [77]:
query = "내 연차는 몇개야?"

In [78]:
result = chain.invoke(query)

{'context': '휴가는  충전이에요 . 눈치  주지  말고 , 눈치  보지  말고 .\n연차는  자율적으로  사용하는  것이  원칙이며 , 팀  내  사전  공유만  잘  하면  됩니다 .\n정시  퇴근도  당연한  권리이며 , 휴가  중인  동료에게는  연락을  삼가는  문화를  지향합니다 .\n“퇴근해 ?ˮ , “ 휴가  좋겠네 ~ˮ 같은  농담도  서로  피합니다 .\n7 .  일을  시작할  땐  ‘ 왜 ʼ 부터  물어요 .\n목적이  분명하면 , 방향도  자연스럽게  생겨요 .\n\n자유롭게  할  수  있습니다 . 회사는  “ 휴가나  퇴근  시  눈치  주는  농담을  하지  않는다 ˮ 는  문\n화를  내재화하여  구성원의  정당한  휴식권을  보장합니다 .\n휴가  및  휴직\n연차휴가 : \xa0법정  연차휴가 (1 년  15 일 , 이후  근속연수에  따라  가산 ) 를  부여하며 , 별도의  승\n인  없이  자유롭게  사용  가능합니다 . 휴가  사용  시  동료에게  업무  인수인계를  완료하고 ,  \n회사  인사관리  시스템으로  최소  1 일  전  신청합니다 . 연차는  시간단위  사용도  허용하며\n\n업무  효율과  워라밸을  지원합니다 .\n유연근무제 : 유연근무제는  다음과  같습니다 . 8 시 ~17 시 , 9 시 ~18 시 , 10 시 ~19 시  중  선택\n하여  출근합니다 . 사원증을  태그하였을  때부터  출근으로  인정됩니다 . 다만 , 분  단위  출근\n은  인정되지  않습니다 . ( 예 : 8 시  39 분  출근  → 9 시  출근으로  인정 )\n초과근무 : 불필요한  야근을  지양하며  업무  효율  향상을  추구합니다 . 업무  특성상  불가피\n\n유연한  휴가  활용을  독려합니다 .\n병가  및  경조휴가 : 직원  본인의  병가 , 가족의  사고  등  긴급한  사유에  대해서는  별도의  유\n급휴가를  제공합니다 . 결혼 , 출산 , 장례  등  경조사  발생  시  경조휴가  

KeyboardInterrupt: 

In [ ]:
print(result.content)

법정 연차휴가는 1년 근무 시 15일이 부여되며, 이후 근속연수에 따라 가산됩니다. 따라서, 당신의 근속연수에 따라 연차가 달라질 수 있습니다. 

근거: "법정 연차휴가 (1 년 15 일 , 이후 근속연수에 따라 가산 ) 를 부여하며"


In [ ]:
query = "2주 뒤에 퇴사하고 싶은데 가능한가? HR에 문의하면 돼?"
result = chain.invoke(query)
print(result.content)

퇴사하고자 할 경우, 인수인계를 고려하여 최소 4주 이전에 퇴사 의사를 통보하는 것을 권장합니다. 따라서 2주 뒤에 퇴사하는 것은 권장되지 않으며, HR에 문의하여 상황을 설명하고 조언을 받는 것이 좋습니다.

근거: 퇴사 통보 부분.
